In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, InputLayer,Conv2D,MaxPool2D,UpSampling2D,\
Conv2DTranspose,Cropping2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint

from Autoencoder import *
from make_video import *

import cv2
import os

1 Physical GPUs, 1 Logical GPUs


In [2]:
p_old=os.getcwd()

In [3]:
p= r'C:\Users\saad\Desktop\Autoencoders\Videos\Results\Bicycle_Frames'
p = p.replace('\\','/')
os.chdir(p)

In [4]:
os.getcwd()

'C:\\Users\\saad\\Desktop\\Autoencoders\\Videos\\Results\\Bicycle_Frames'

In [5]:
path = r'C:\Users\saad\Desktop\Autoencoders\Videos\Dataset'
vid_name = 'Bicycle_Sequence.mp4'

In [6]:
names,fps,width,height=read_video(os.getcwd(),os.path.join(path,vid_name))

Loading Video...
Completed


In [7]:
print(f'The fps is {fps}, The width of the frame is {width} and the height of the frame is {height}')

The fps is 25.0, The width of the frame is 3840 and the height of the frame is 2160


In [ ]:
names

In [8]:
images = np.array(read_imgs(os.getcwd(),names,2,640,480))
images = images.astype('float32')/255.
print(f'The shape of the images is {images.shape}')

The shape of the images is (224, 480, 320, 3)


In [9]:
img_shape = images.shape[1:]
print(img_shape)

(480, 320, 3)


In [10]:
model1,model2 = build_autoencoder(img_shape,512,'Hi-Res',None)

In [11]:
weights = r'C:\Users\saad\Desktop\Autoencoders\Videos\Results\Weights'
weights = weights.replace('\\','/')


In [12]:
model1.load_weights(os.path.join(weights,'model1.hdf5'))
model2.load_weights(os.path.join(weights,'model2.hdf5'))

In [13]:
img_new = model1.predict(images)

In [14]:
img_new.shape

(224, 480, 320, 3)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img_new[75,:,:,:])

In [15]:
from tqdm import tqdm
import numpy as np
preds=[]
for i in tqdm(range(len(img_new))):
    preds.append(model2.predict(np.expand_dims(img_new[i],axis=0)))

100%|████████████████████████████████████████████████████████████████████████████████| 224/224 [00:08<00:00, 26.87it/s]


In [16]:
pr = np.array(preds)
pr.shape

(224, 1, 480, 320, 3)

In [17]:
pr= pr.reshape(224,480,320,3)

In [ ]:
plt.imshow(pr[75,:,:,:])

In [ ]:
plt.imshow(images[75,:,:,:])

In [ ]:
im = 3
plt.figure(figsize=(14,10))

plt.subplot(131)
plt.imshow(images[im,:,:,:])
plt.title('Original')

plt.subplot(132)
plt.imshow(img_new[im,:,:,:])
plt.title('Intermediate')

plt.subplot(133)
plt.imshow(pr[im,:,:,:])
plt.title('Reconstructed')

plt.show()

In [18]:
predictions = pr * 255.

In [19]:
VIDEO = r'C:\Users\saad\Desktop\Autoencoders\Videos\Results\Video'
video_creater('Recon_bicycle.avi',VIDEO,25.,640,480,predictions) 

Frames Saving:  10%|██████▋                                                          | 23/224 [00:00<00:00, 223.17it/s]

Working on Reconstructed Video


Concatenating Frames: 100%|██████████████████████████████████████████████████████████| 112/112 [00:01<00:00, 78.97it/s]


Reconstructed Frames removed.


In [20]:
 original_video('Original_bicycle.avi',VIDEO,os.getcwd(),25.,names)

Frame Reading:  21%|█████████████▉                                                   | 24/112 [00:00<00:00, 236.36it/s]

Working on Original Video


Frame Reading: 100%|████████████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 237.36it/s]


In [ ]:
os.getcwd()